In [31]:
%reset -f

import numpy as np
import numba as  nb
import pandas as pd

all = pd.read_parquet('tmp.parq')
subset = all[-10000:]

In [32]:
@nb.jit(nopython=True)
def do_a_manual(t:np.array, bid:np.array, ask:np.array, bin_start:np.array, ) -> np.array:
    n_ticks, n_bins = len(t), len(np.unique(bin_start))
    i, j, ii = 0, 0, 0
    bin_index = np.full(n_bins, 0)
    n_samples = np.full(n_bins, 0)
    o = np.full(n_bins, 0.0)
n_samples
    while j < n_ticks:
        while (j < n_ticks) & (bin_start[i] == bin_start[j]):
            j += 1
        # i is the index of the first element on the binned timestamp
        # je is the index to the last element of the binned timestamp
        # ii is the insertion point for the current bin

        je = j - 1
        bin_index[ii] = bin_start[je]
        n_samples[ii] = j - i
        ii += 1
        i = j
            
    return bin_index, n_samples

#pad_on_bin_bopundary
def foo(ticks:pd.DataFrame,
        bin_size_sec:int,
        time_col:str = "t",
        ) -> pd.DataFrame:

    tt = ticks[time_col].values.astype('datetime64[ms]').astype(np.int64)
    bin_start = ticks[time_col].values.astype(f'datetime64[{bin_size_sec}s]').astype('datetime64[ms]').astype(np.int64)
    bin_index, n_samples = do_a_manual(ticks.t.values, ticks.bid.values, ticks.ask.values, bin_start)
    return pd.DataFrame({'bin_start':bin_index, 'n_samples':n_samples})

res = foo(subset, 60)
res[:5]

,bin_start,n_samples
0,1687391820000,10
1,1687391880000,14
2,1687391940000,14
3,1687392000000,43
4,1687392060000,50


In [35]:
res = foo(all, 60)
res[:5]

,bin_start,n_samples
0,1686079320000,18
1,1686079380000,7
2,1686079620000,4
3,1686079740000,3
4,1686083220000,6


In [ ]:
# def make_bin(x):
#     df = pd.Series()
#     t = x['t'].values
#     bid, ask, bsz, asz = x.bid.values, x.ask.values, x.bsz.values, x.asz.values
#     bin_end = x['bin_end'].values[0]
#     mid  = (bid + ask)/2.0
#     dt = np.diff(np.append(t, bin_end))


#     df['bin_end'] = bin_end 
#     df['o'] = mid[0]
#     df['h'] = np.max(mid)
#     df['l'] = np.min(mid)
#     df['c'] = mid[-1]
#     df['twap'] = np.sum(mid*dt)/np.sum(dt)
#     df['mean'] = np.mean(mid)

#     return df

# def gen_binned_data(ticks:pd.DataFrame,
#                     bin_size_sec:int,
#                     time_col:str = "t",
#                     ) -> pd.DataFrame:

#     # this is needed for twaps to work correctlly, otherwise we see downward bias in twaps
#     ticks[time_col] = ticks[time_col].values.astype('datetime64[ns]')

#     ticks['bin_start'] = ticks[time_col].values.astype(f'datetime64[{bin_size_sec}s]').astype('datetime64[s]')
#     ticks['bin_end'] = ticks['bin_start'].values + np.timedelta64(bin_size_sec, 's')

#     padded = ticks.groupby(['bin_start'], as_index=False).apply(make_bin).reset_index()
#     return padded

# subset = bid_ask[bid_ask.t > np.datetime64('2023-06-19 15:23:00')].copy()
# x = gen_binned_data(subset, 60)
# x

In [90]:
annotated['bin_start'].values

array(['2023-06-06T19:22:00.000000000', '2023-06-06T19:22:00.000000000',
       '2023-06-06T19:22:00.000000000', ...,
       '2023-06-20T18:03:00.000000000', '2023-06-20T18:03:00.000000000',
       '2023-06-20T18:03:00.000000000'], dtype='datetime64[ns]')

,t,bid,ask,bsz,asz
0,2023-06-06 19:22:24.585,1.16200,1.16237,1.16200,1.16237
1,2023-06-06 19:22:25.612,1.16201,1.16238,1.16201,1.16238
2,2023-06-06 19:22:31.087,1.16202,1.16239,1.16202,1.16239
3,2023-06-06 19:22:32.525,1.16201,1.16238,1.16201,1.16238
4,2023-06-06 19:22:34.189,1.16200,1.16237,1.16200,1.16237
...,...,...,...,...,...
196263,2023-06-20 06:48:39.465,1.16954,1.16991,1.16954,1.16991
196264,2023-06-20 06:48:40.766,1.16952,1.16989,1.16952,1.16989
196265,2023-06-20 06:48:41.801,1.16950,1.16987,1.16950,1.16987
196266,2023-06-20 06:48:42.850,1.16950,1.16987,1.16950,1.16987
